In [3]:
import pandas as pd
import numpy as np
import os
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [4]:
file_list = set(os.listdir('./data/'))

In [5]:
# The GraphMl is a easy to use file format for graphs: In our case, mixed directed graphs.

# check to see if the uberspeeds.graphml has already been created, if not, make it
# Uberspeeds will be the final final that'll be produced. 
if 'uberspeeds.graphml' in file_list:
    london_graph = ox.save_load.load_graphml('uberspeeds.graphml',folder='data')
    
else:
    # import segments look-up, available from Uber Movements
    # This dataset provides a mapping between Movement's segment IDs and OSM's way IDs.
    segments = pd.read_csv('./data/movement-segments-to-osm-ways-london-2019.csv')

    # import output from sql query 
    uber_df = pd.read_csv('../data/csv/xaa.csv')

    uber_df = uber_df.merge(segments, how='left', on='segment_id')

    uber_summary = uber_df[['osm_way_id','speed_mph_mean','speed_mph_stddev']].groupby(['osm_way_id'], as_index=False).sum()
    uber_summary['speed'] = uber_summary['speed_mph_mean'] / uber_summary['speed_mph_stddev']
    
    # Creating the outline for the city. GeoJSON or shapefiles work best.
    # Check to see if the shape for the city exists if not create it.
    if 'london.graphml' in file_list:
        london_graph = ox.save_load.load_graphml('london.graphml',folder='data')
    else:
        # You can use ox to load from a shapefile but for me loading geometries directly did the job.
        # which_result=2 would make it such that london_graph expects a polygon
        london_graph = ox.graph_from_place('London, England', network_type='drive', which_result=2)
        ox.save_load.save_graphml(london_graph, filename='london.graphml', folder='data')
    
    for from_node, to_node, edge in london_graph.edges(data=True):
        
        # first we'll iterate through the edge id's to determine the associated ubers speed
        osm_id = edge['osmid']
        
        if type(osm_id) == list:
            
            uber_speed = list() 
            for element in osm_id:            
                uber_speed.append(uber_summary['speed'][uber_summary['osm_way_id'] == element].mean())
            
            # calculate average speed from the speed array, dropping any NaN values
            uber_speed = np.nanmean(np.array(uber_speed))
        
        else:
            uber_speed = uber_summary['speed'][uber_summary['osm_way_id'] == osm_id].mean()
            
            
        # now let's iterate through the posted speed limit(s)
        try:    
            speeds = edge['maxspeed']
        except KeyError:
            speeds = str('unknown')
            
        print("Posted speed limits" + speeds)
        
        
        if type(speeds) == list:
            speed_lim = list()
            
            for speed in speeds:
                speed_lim.append(pd.to_numeric(speed.split(' ')[0], errors=np.nan, downcast='float'))
            
            speed_lim = np.nanmax(np.array(speed_lim))
        else:
            speed_lim = pd.to_numeric(speeds.split(' ')[0], errors=np.nan, downcast='float')
        
        # add to edge data
        # calculate speed-to-limit ratio
        diff = uber_speed / speed_lim
        # assign linewidth for different ratios
        lw = 2 if 1 <= diff < 1.25 else 3 if diff >= 1.25 else 1
        # assign different line colour for different ratios
        colour = '#ffbf00' if 1 <= diff < 1.25 else '#99cc99' if diff < 1 else '#f60404' if diff >= 1.25 else '#c6c7be'
        
        # add back into graph
        london_graph[from_node][to_node][0]['uberspeedmph'] = uber_speed
        london_graph[from_node][to_node][0]['speedint'] = speed_lim
        london_graph[from_node][to_node][0]['diff'] = uber_speed / speed_lim
        london_graph[from_node][to_node][0]['colours'] = colour
        london_graph[from_node][to_node][0]['lw'] = lw
        
        print ('The code has reached here')


FileNotFoundError: [Errno 2] File b'../data/movement-segments-to-osm-ways-london-2019.csv' does not exist: b'../data/movement-segments-to-osm-ways-london-2019.csv'

In [18]:
# copy and run this in the console to get a pop-up window:- %matplotlib qt5
# you can then save specific plots from there. If you know where to look, you could just set plot limits instead (again, cba)
# set edge colors
nodes, edges = ox.graph_to_gdfs(sf_graph)
edges.oneway = edges.oneway.apply(lambda x: int(x == 'true'))
edges_lite = edges[['colours','diff','geometry','speedint','u','v','uberspeedmph']]
edges_lite.to_file('../data/uberspeeds.csv', driver='csv') # save for later

NameError: name 'sf_graph' is not defined

In [14]:
ec = pd.DataFrame([data for u, v, data in sf_graph.edges(data=True)])
ec['colours'][ec.colours.isna()] = '#c6c7be'
ec['lw'][ec.lw.isna()] = 1

fig, ax = ox.plot_graph(sf_graph, edge_color=ec['colours'], edge_linewidth=ec['lw'], 
                        node_size=1, node_zorder=1, fig_height=16, dpi=420)

NameError: name 'sf_graph' is not defined

In [6]:
df = gpd.read_file('../seattletester1.geojson')

In [14]:
print(df.head(5))

    osmhighway  osmwayid  osmstartnodeid  osmendnodeid osmname osmoneway  \
0      service   4644025       687818172     687817976    None      None   
1      service   4644025       687818172     687817976    None      None   
2  residential   6318471        52978794     349922585    None      None   
3  residential   6318471        52978794     349922585    None      None   
4  residential   6325880        53011616      53011628    None      None   

                                            geometry  
0  LINESTRING (-122.05470 37.41488, -122.05495 37...  
1  LINESTRING (-122.05573 37.41574, -122.05568 37...  
2  LINESTRING (-122.05231 37.59350, -122.05074 37...  
3  LINESTRING (-122.05062 37.59279, -122.05073 37...  
4  LINESTRING (-121.87757 37.60665, -121.87911 37...  


In [7]:
from keplergl import KeplerGl
map_2 = KeplerGl(height=400, data={"data_1": df})
map_2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'data_1':          osmhighway   osmwayid  osmstartnode  osmendnode  \
0       residential    61…